In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
# One-hot encoding categorical columns
dataset = pd.get_dummies(dataset, drop_first=True)


In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts() # Here we have to count the o's and 1's 
#Here we can check this BALANCED/IMBALANCED?
#If Balanced Dataset = It'll be in Equal numbers for both 0 & 1
# If Imbalanced = Deviations will be there.
# Unique Entries using .value_counts() method

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[['Age','EstimatedSalary','Purchased','Gender_Male']]
dep=dataset["Purchased"] # Here as usual we are not splitting the training set and test for Regression - GRID purpose.
#But here in the classification Grid we should spilt the training set and test set.
#If we not split the same Report will generate.  

In [9]:
indep.shape

(400, 4)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
# Split into training set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
#Here We should add the Standard Scaler option as well

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() #if our predcitions will be under -2 to +2, Then we can keep it in the Standard Scaler
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
# htpps://scikit-learn.org/stable/modules/model_evaluation.html#storing-parameter

param_grid = {
    'kernel':['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma':['auto','scale'],
    'C':[10,100,1000,2000,3000]
}
# ROC => SVC(probability=True) We should give this probability then only We can do the 'ROC_AUC'
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, cv=5, n_jobs = -1, scoring = 'f1_weighted') #It will run automatically and take the best model from the grid list
# We can do with RECALL & PRecision as well for the prediction(Which model is giving better performance)
# fitting the model for grid search
grid.fit(X_train, y_train)


Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [15]:
# Print the best parameter after tuning
# print (grid.best_params_)
re=grid.cv_results_
# print(re)
grid_predictions = grid.predict(X_test) # we can store the X_test in to gird.prediction

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions, average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_), f1_macro)

The f1_macro value for best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'linear'}: 1.0


In [17]:
print("The Confusion Matrix:\n", cm)

The Confusion Matrix:
 [[85  0]
 [ 0 49]]


In [18]:
print("The Report:\n", clf_report)

The Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        85
           1       1.00      1.00      1.00        49

    accuracy                           1.00       134
   macro avg       1.00      1.00      1.00       134
weighted avg       1.00      1.00      1.00       134



In [19]:
from sklearn.metrics import roc_auc_score # [ROC = Receiver Operating Charactersitic Curve], [AUC = Area Under Curve]
roc_auc_score(y_test, grid.predict_proba(X_test)[:,1]) # proba => Parabola.
# where [:,1] => All rows in the first column
# : => represents the rows [We can take entire rows]
# And the 1 => Represents Column [first columns] 

np.float64(1.0)

In [20]:
table = pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003077,0.000159,0.003611,0.000257,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.006176,0.001157,0.003747,0.000515,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.003663,0.000382,0.003920,0.000764,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.002627,0.000476,0.003811,0.001452,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.002829,0.001037,0.003667,0.000871,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
5,0.005117,0.000533,0.003371,0.000516,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
6,0.004251,0.001201,0.003249,0.000614,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
7,0.002565,0.000647,0.003522,0.000964,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
8,0.002163,0.000436,0.003086,0.000337,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
9,0.004885,0.000580,0.003457,0.000748,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


In [23]:
UserID_input=int(input("UserID:"))
Age_input=float(input("age:"))
EstimatedSalary_input=float(input("Estimated Salary:"))
Gender_Male_input=int(input("Gender Male 0 or 1:"))

UserID: 15691863
age: 46
Estimated Salary: 41000
Gender Male 0 or 1: 1


In [30]:
Future_Prediction=grid.predict([[UserID_input, Age_input, EstimatedSalary_input, Gender_Male_input]]) # Change the parameter according to your dataset
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
